# Батч нормализация

In [ ]:
model = keras.models.Sequental([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

# Использование предобученной модели. transfer learning

In [ ]:
model_A = keras.models.load_model("model_A.h5") # Или .keras
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid")) # удаление многоклассового выхода и добавление бинарного

# Для того чтобы модель А не изменялась(веса), необходимо клонировать модель

In [ ]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

Из-за того, что новый слой был инициализирован случаным образом, необходимо заморозимть слои для узнавания приемлемых весов

In [ ]:
for layer in model_B_on_A.layers[:-1]:
  layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

При разморозке весов, lr уменьшается, чтобы сильно не изменять веса модели

In [ ]:
for layer in model_B_on_A.layers[:-1]:
  layer.trainable = True

optimizer = keras.optimizers.SGD(lr=1e-4) # старая версия, смотреть 7 тетрадь
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))

Оценка. Transfer learning лучше работает, при CNN

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

# Оптимизаторы GD

In [ ]:
# Momentum optimization of GD
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)

# Nesterov Accelerated Gradient | NAG
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

# AdaGrad

# RMSProp
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)

# Adam
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

# AdaMax - Заменяет норму L2, на Норму L(inf), использует максимальное затухание градиентов во времени
# Nadam - оптимизация Adam + трюк Нестерова

# Оптимизация lr, во время обучения

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)

def exponentioal_decay_fn(epoch):
  return 0.01 * 0.1**(epoch/20)

# если не хотим жёстко кодировать n0 и s
def exponential_decay(lr0, s):
  def exponentioal_decay_fn(epoch):
    return lr0 * 0.1**(epoch / s)
  return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=20)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(..., callbacks=[lr_scheduler])

# кусочно-линейный постоянный график
def piecewise_constant_fn(epoch):
  if epoch < 5:
    return 0.01
  elif epoch < 15:
    return 0.005
  else:
    return 0.001

lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

# обновление lr на каждом шаге
s = 20 * len(X_train) // 32

learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.SGD(learning_rate)

# Регуляризация L1, L2

In [ ]:
layer = keras.layers.Densse(100, activation="elu", kernel_initializer="he_normal", kernel_regularizer=keras.regularizers.l2(0.01))

# keras.regularizers.l1()
# keras.regularizers.l1_l2() 2 коэффицента регуляризации

In [ ]:
from Functools import partial

RegularizedDense = partial(
    keras.layers.Dense,
    activation="elu",
    kernel_initializer="he_normal",
    kernel_regularizer=keras.regularizers.l2(0.01)
)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax", kernel_initializer="glorot_uniform")
])

# Dropout

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2), # rate = proba %
    RegularizedDense(300),
    keras.layers.Dropout(rate=0.2),
    RegularizedDense(100),
    keras.layers.Dropout(rate=0.2),
    RegularizedDense(10, activation="softmax", kernel_initializer="glorot_uniform")
])

# MC Dropout | Monte Carlo Dropout

In [ ]:
y_probas = np.stack([model(X_test_scaled, training=True) for sample in range(100)]) # [100, 10 000, 10]
y_proba = y_proba.mean(axis=0) # [10 000, 10]

In [ ]:
class MCDropout(keras.layers.Dropout):
  def call(self, inputs):
    return super().call(inputs, training=True)

# Max Norm

In [ ]:
keras.layers.Dense(
    100, activation="elu",
    kernel_initializer="he_normal",
    kernel_constraints.max_norm(1.)
)